In [ ]:
pip install -r requirements.txt


In [6]:
customers = pd.read_csv('customers.csv')
noncustomers = pd.read_csv('noncustomers.csv')
usage_actions = pd.read_csv('usage_actions.csv')

In [7]:
# Create new column for concatenation
customers['is_customer']=1
noncustomers['is_customer'] = 0
print(customers.head())
print(noncustomers.head())

    CLOSEDATE     MRR  ALEXA_RANK EMPLOYEE_RANGE               INDUSTRY   id  \
0  2019-06-20  290.00      309343    201 to 1000                  Other  199   
1  2019-03-22  392.54    16000001      51 to 200                    NaN  147   
2  2020-01-08  -61.15       60502  1001 to 10000       HIGHER_EDUCATION  118   
3  2019-09-19  400.00     3575321    201 to 1000      CONSUMER_SERVICES  126   
4  2019-01-27  209.98      273063       26 to 50  Technology - Software  174   

   is_customer  
0            1  
1            1  
2            1  
3            1  
4            1  
   ALEXA_RANK EMPLOYEE_RANGE              INDUSTRY    id  is_customer
0   4489573.0       11 to 25                   NaN  4916            0
1   4027013.0        6 to 10                   NaN  3489            0
2  16000001.0       26 to 50     COMPUTER_SOFTWARE  2813            0
3    814822.0  1001 to 10000  HOSPITAL_HEALTH_CARE   727            0
4  16000001.0       26 to 50                   NaN  2166           

In [8]:
# Combine customer and noncustomer data. Drop revenue and close date
comb_cust = pd.concat([customers.drop(columns =['MRR','CLOSEDATE']),
                       noncustomers],axis = 0)

comb_cust.tail()

,ALEXA_RANK,EMPLOYEE_RANGE,INDUSTRY,id,is_customer
4998,16000001.0,NaN,NaN,637,0
4999,20183.0,1001 to 10000,Non-Profit/Educational Institution,4921,0
5000,16000001.0,6 to 10,NaN,1215,0
5001,NaN,11 to 25,NaN,2693,0
5002,16000001.0,2 to 5,NaN,208,0


In [9]:
#Alexa Rank also has a few nulls, so we will fill in with the median
comb_cust['ALEXA_RANK'] = comb_cust['ALEXA_RANK'].fillna(comb_cust['ALEXA_RANK'].median())

In [10]:
comb_cust['is_customer'].value_counts()

is_customer
0    5003
1     200
Name: count, dtype: int64

In [11]:
#3/4 of Industry designations are also null. We will drop.
print(comb_cust['INDUSTRY'].isna().sum()/len(comb_cust))
comb_cust.drop('INDUSTRY', axis=1, inplace=True)

0.7407265039400346


In [12]:
print(comb_cust.dtypes)
comb_cust.isna().sum()

ALEXA_RANK        float64
EMPLOYEE_RANGE     object
id                  int64
is_customer         int64
dtype: object


ALEXA_RANK          0
EMPLOYEE_RANGE    534
id                  0
is_customer         0
dtype: int64

In [13]:
# We convert to dummy variables to measure employee size
emp = pd.get_dummies(comb_cust['EMPLOYEE_RANGE'])
cust_emp = pd.concat([comb_cust,emp], axis =1)
cust_emp.drop('EMPLOYEE_RANGE', axis=1, inplace=True)
cust_emp.tail()

,ALEXA_RANK,id,is_customer,1,"10,001 or more",1001 to 10000,11 to 25,2 to 5,201 to 1000,26 to 50,51 to 200,6 to 10
4998,16000001.0,637,0,False,False,False,False,False,False,False,False,False
4999,20183.0,4921,0,False,False,True,False,False,False,False,False,False
5000,16000001.0,1215,0,False,False,False,False,False,False,False,False,True
5001,16000001.0,2693,0,False,False,False,True,False,False,False,False,False
5002,16000001.0,208,0,False,False,False,False,True,False,False,False,False


In [14]:
# Merge combined customer data sets with customer actions
# Remember non-customers can perform actions
# Use a left join with usage_actions the left table, because we are 
# analyzing customer actions.
cust_actions = pd.merge(usage_actions, cust_emp, how = 'left', on = 'id')
cust_actions.tail()

,WHEN_TIMESTAMP,ACTIONS_CRM_CONTACTS,ACTIONS_CRM_COMPANIES,ACTIONS_CRM_DEALS,ACTIONS_EMAIL,USERS_CRM_CONTACTS,USERS_CRM_COMPANIES,USERS_CRM_DEALS,USERS_EMAIL,id,...,is_customer,1,"10,001 or more",1001 to 10000,11 to 25,2 to 5,201 to 1000,26 to 50,51 to 200,6 to 10
25385,2019-12-16 00:00:00.000,2,2,0,0,1,1,0,0,406,...,0,False,False,False,False,False,False,False,True,False
25386,2019-08-05 00:00:00.000,0,0,0,0,0,0,0,0,1885,...,0,False,False,False,False,False,False,False,False,True
25387,2020-02-24 00:00:00.000,84,286,151,0,8,8,9,0,51,...,1,False,False,False,True,False,False,False,False,False
25388,2020-02-03 00:00:00.000,85,0,24,0,1,0,1,0,2360,...,0,False,False,False,False,True,False,False,False,False
25389,2020-06-08 00:00:00.000,0,0,0,0,0,0,0,0,4565,...,0,False,False,False,False,False,False,False,False,False


In [15]:
cust_actions['date'] = pd.to_datetime(cust_actions['WHEN_TIMESTAMP']).dt.date
cust_actions.drop(['WHEN_TIMESTAMP'], axis=1, inplace=True)

In [16]:
cust_actions.head()

,ACTIONS_CRM_CONTACTS,ACTIONS_CRM_COMPANIES,ACTIONS_CRM_DEALS,ACTIONS_EMAIL,USERS_CRM_CONTACTS,USERS_CRM_COMPANIES,USERS_CRM_DEALS,USERS_EMAIL,id,ALEXA_RANK,...,1,"10,001 or more",1001 to 10000,11 to 25,2 to 5,201 to 1000,26 to 50,51 to 200,6 to 10,date
0,278,0,34,0,2,0,2,0,616,16000001.0,...,False,False,False,False,True,False,False,False,False,2020-06-01
1,0,0,0,0,0,0,0,0,3059,16000001.0,...,False,False,False,False,False,False,False,False,True,2020-04-20
2,210,1,17,11,3,1,4,1,2813,16000001.0,...,False,False,False,False,False,False,True,False,False,2019-04-29
3,1,1,1,0,1,1,1,0,5076,16000001.0,...,False,False,False,False,True,False,False,False,False,2020-02-24
4,9,9,7,0,1,1,1,0,3983,16000001.0,...,False,False,False,False,False,False,False,False,True,2020-02-17


In [17]:
cust_actions['is_customer'].value_counts()

is_customer
0    16926
1     8464
Name: count, dtype: int64

In [18]:
cust_id_cust = cust_actions[['id','is_customer']]
cust_id_cust.drop_duplicates(inplace = True)

C:\Users\matth\AppData\Local\Temp/ipykernel_45536/2132336410.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [19]:
cust_id_cust['is_customer'].value_counts()

is_customer
0    3370
1     199
Name: count, dtype: int64

In [20]:
# Outlier detection - very few companies have more than 100 users in the
# system
cust_actions.query('USERS_CRM_CONTACTS> 100').sort_values('date')

,ACTIONS_CRM_CONTACTS,ACTIONS_CRM_COMPANIES,ACTIONS_CRM_DEALS,ACTIONS_EMAIL,USERS_CRM_CONTACTS,USERS_CRM_COMPANIES,USERS_CRM_DEALS,USERS_EMAIL,id,ALEXA_RANK,...,1,"10,001 or more",1001 to 10000,11 to 25,2 to 5,201 to 1000,26 to 50,51 to 200,6 to 10,date
17476,20740,13,23,5,376,9,14,1,119,1740005.0,...,False,False,False,False,False,True,False,False,False,2020-05-04
19358,27936,4,12,4,314,3,10,1,119,1740005.0,...,False,False,False,False,False,True,False,False,False,2020-05-11
14728,24126,1,61,3,309,1,9,1,119,1740005.0,...,False,False,False,False,False,True,False,False,False,2020-05-18
20874,20539,0,4,0,288,0,3,0,119,1740005.0,...,False,False,False,False,False,True,False,False,False,2020-05-25
4226,21066,0,6,4,283,0,3,2,119,1740005.0,...,False,False,False,False,False,True,False,False,False,2020-06-15
15929,17718,0,3,1,270,0,2,1,119,1740005.0,...,False,False,False,False,False,True,False,False,False,2020-06-29
661,21982,3,35,0,276,2,6,0,119,1740005.0,...,False,False,False,False,False,True,False,False,False,2020-07-06
8117,24551,0,16,0,264,0,5,0,119,1740005.0,...,False,False,False,False,False,True,False,False,False,2020-07-13
22232,25975,1,4,3,270,1,2,1,119,1740005.0,...,False,False,False,False,False,True,False,False,False,2020-07-20
20124,22557,1,11,0,253,1,2,0,119,1740005.0,...,False,False,False,False,False,True,False,False,False,2020-07-27


In [21]:
# Outlier detection. Very few cos perform greater than 10k actions
cust_actions.query('ACTIONS_CRM_CONTACTS> 10000').sort_values('date')

,ACTIONS_CRM_CONTACTS,ACTIONS_CRM_COMPANIES,ACTIONS_CRM_DEALS,ACTIONS_EMAIL,USERS_CRM_CONTACTS,USERS_CRM_COMPANIES,USERS_CRM_DEALS,USERS_EMAIL,id,ALEXA_RANK,...,1,"10,001 or more",1001 to 10000,11 to 25,2 to 5,201 to 1000,26 to 50,51 to 200,6 to 10,date
16402,10037,0,10,2,12,0,6,1,39,2392953.0,...,False,False,False,False,False,True,False,False,False,2019-01-07
14620,10659,0,16,9,11,0,3,2,39,2392953.0,...,False,False,False,False,False,True,False,False,False,2019-01-14
17476,20740,13,23,5,376,9,14,1,119,1740005.0,...,False,False,False,False,False,True,False,False,False,2020-05-04
19358,27936,4,12,4,314,3,10,1,119,1740005.0,...,False,False,False,False,False,True,False,False,False,2020-05-11
14728,24126,1,61,3,309,1,9,1,119,1740005.0,...,False,False,False,False,False,True,False,False,False,2020-05-18
20874,20539,0,4,0,288,0,3,0,119,1740005.0,...,False,False,False,False,False,True,False,False,False,2020-05-25
4226,21066,0,6,4,283,0,3,2,119,1740005.0,...,False,False,False,False,False,True,False,False,False,2020-06-15
15929,17718,0,3,1,270,0,2,1,119,1740005.0,...,False,False,False,False,False,True,False,False,False,2020-06-29
661,21982,3,35,0,276,2,6,0,119,1740005.0,...,False,False,False,False,False,True,False,False,False,2020-07-06
8117,24551,0,16,0,264,0,5,0,119,1740005.0,...,False,False,False,False,False,True,False,False,False,2020-07-13


In [ ]:
cust_actions.to_pickle("data/df1.pkl")